In [ ]:
import tensorflow as tf 
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sentiment=['noVideo','amusing-1','amusing-2','boring-1','boring-2','relaxed-1','relaxed-2','scary-1','scary-2','noVideo','startVid','bluVid','endVid']

In [ ]:
def choose_subjects(Number):

    annotations_df = pd.read_csv('/content/drive/MyDrive/Anno/sub_1.csv')
    physiological_df = pd.read_csv('/content/drive/MyDrive/Physio/sub_1.csv')
    for i in range(2,Number+1) :
        chemin1='/content/drive/MyDrive/Anno/sub_{}.csv'.format(i)
        chemin2='/content/drive/MyDrive/Physio/sub_{}.csv'.format(i)
        annotations_df = pd.concat([annotations_df,pd.read_csv(chemin1)],axis=0)
        physiological_df = pd.concat([physiological_df,pd.read_csv(chemin2)],axis=0)
    return annotations_df,physiological_df

In [ ]:
anno_df,physio_df=choose_subjects(3)

In [ ]:
anno_df

,jstime,valence,arousal,video
0,0,5.0,5.0,10
1,50,5.0,5.0,10
2,100,5.0,5.0,10
3,150,5.0,5.0,10
4,200,5.0,5.0,10
...,...,...,...,...
49027,2451350,5.0,5.0,12
49028,2451400,5.0,5.0,12
49029,2451450,5.0,5.0,12
49030,2451500,5.0,5.0,12


In [ ]:
physio_df

,daqtime,ecg,bvp,gsr,rsp,skt,emg_zygo,emg_coru,emg_trap,video
0,0,0.846,38.172,11.749,27.610,29.495,7.288,8.684,6.550,10
1,1,0.836,38.249,11.737,27.601,29.488,7.288,8.725,6.468,10
2,2,0.843,38.058,11.757,27.600,29.488,7.329,8.726,6.589,10
3,3,0.836,38.075,11.730,27.572,29.481,7.207,8.644,6.468,10
4,4,0.852,38.210,11.745,27.591,29.484,7.247,8.765,6.507,10
...,...,...,...,...,...,...,...,...,...,...
2451579,2451579,0.925,35.899,31.804,30.417,24.110,18.049,6.960,6.262,12
2451580,2451580,0.912,36.059,31.696,30.427,24.113,18.170,7.082,6.302,12
2451581,2451581,0.905,36.214,31.734,30.446,24.117,18.212,7.123,6.343,12
2451582,2451582,0.882,36.254,31.738,30.437,24.106,17.968,7.002,6.262,12


In [ ]:
def select_rows_join_on_time(i):
    if i%50==0:
        return True
    else:
        return False

In [ ]:
physio_timejoint_df=physio_df.iloc[[select_rows_join_on_time(i) for i in range(len(physio_df))]]
physio_timejoint_df.reset_index(inplace=True,drop=True)
anno_df.reset_index(inplace=True ,drop=True)
df1 = pd.concat([physio_timejoint_df,anno_df],axis=1)

In [ ]:
def select_rows_median(df):
    df=df.iloc[:,1:-1]
    rolling_median = df.rolling(window=50, min_periods=1).median()
    result = pd.DataFrame({'ecg': rolling_median['ecg'][49::50].values, 
                        'bvp': rolling_median['bvp'][49::50].values, 
                        'gsr': rolling_median['gsr'][49::50].values,
                        'rsp': rolling_median['rsp'][49::50].values,
                        'skt': rolling_median['skt'][49::50].values,
                        'emg_zygo': rolling_median['emg_zygo'][49::50].values,
                        'emg_coru': rolling_median['emg_coru'][49::50].values,
                        'emg_trap': rolling_median['emg_trap'][49::50].values})
    last_median = df.tail(34).median()
    result = result.append(last_median.to_dict(), ignore_index=True)
    return result

In [ ]:
physio_median_df = select_rows_median(physio_df)

<ipython-input-10-2bf4dd871938>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(last_median.to_dict(), ignore_index=True)


In [ ]:
def select_rows_mean(df):
    df=df.iloc[:,1:-1]
    rolling_median = df.rolling(window=50, min_periods=1).mean()
    result = pd.DataFrame({'ecg': rolling_median['ecg'][49::50].values, 
                        'bvp': rolling_median['bvp'][49::50].values, 
                        'gsr': rolling_median['gsr'][49::50].values,
                        'rsp': rolling_median['rsp'][49::50].values,
                        'skt': rolling_median['skt'][49::50].values,
                        'emg_zygo': rolling_median['emg_zygo'][49::50].values,
                        'emg_coru': rolling_median['emg_coru'][49::50].values,
                        'emg_trap': rolling_median['emg_trap'][49::50].values})
    last_mean = df.tail(34).mean()
    result = result.append(last_mean.to_dict(), ignore_index=True)
    return result

In [ ]:
physio_mean_df = select_rows_mean(physio_df)

<ipython-input-12-e4b5e5ebaf7b>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(last_mean.to_dict(), ignore_index=True)


In [ ]:
visual_df = pd.read_csv('/content/drive/MyDrive/Physio/sub_1.csv')
df_subset_ecg = visual_df.iloc[:,1]
df_subset_bvp = visual_df.iloc[:,2]
df_subset_gsr = visual_df.iloc[:, 3]
df_subset_rsp = visual_df.iloc[:,4]
df_subset_skt = visual_df.iloc[:,:5]
df_subset_emg_zygo = visual_df.iloc[:,6]
df_subset_emg_coru = visual_df.iloc[:, 7]
df_subset_emg_trap = visual_df.iloc[:, 8]

trace1 = go.Scatter(x=df_subset_ecg.index, y=df_subset_ecg.values, mode='lines', name='ecg', line=dict(color='blue'))
trace2 = go.Scatter(x=df_subset_bvp.index, y=df_subset_bvp.values, mode='lines', name='bvp', line=dict(color='green'))
trace3 = go.Scatter(x=df_subset_gsr.index, y=df_subset_gsr.values, mode='lines', name='gsr', line=dict(color='red'))
trace4 = go.Scatter(x=df_subset_rsp.index, y=df_subset_rsp.values, mode='lines', name='rsp', line=dict(color='purple'))
trace5 = go.Scatter(x=df_subset_skt.index, y=df_subset_skt.values, mode='lines', name='skt', line=dict(color='grey'))
trace6 = go.Scatter(x=df_subset_emg_zygo.index, y=df_subset_emg_zygo.values, mode='lines', name='emg_zygo', line=dict(color='brown'))
trace7 = go.Scatter(x=df_subset_emg_coru.index, y=df_subset_emg_coru.values, mode='lines', name='emg_coru', line=dict(color='yellow'))
trace8 = go.Scatter(x=df_subset_emg_trap.index, y=df_subset_emg_trap.values, mode='lines', name='emg_trap', line=dict(color='orange'))

layout = go.Layout(title='Figure des signaux ECG', xaxis=dict(title='temps'), yaxis=dict(title='Signal'))

fig = go.Figure(data=[trace1, trace2, trace3, trace4,trace5, trace6, trace7, trace8], layout=layout)

fig.show()

In [ ]:
df1

In [ ]:
features_df = df1.iloc[:,1:8]
features_scaled=features_df.values
#features_scaled = features_scaled.reshape(-1,1) #uncomment if using 1 feature
min_max_scaler = sk.preprocessing.MinMaxScaler()
features_scaled=min_max_scaler.fit_transform(features_scaled)

target_df=df1.iloc[:,11]
target=target_df.values

In [ ]:
x_Train, x_Test, y_Train, y_Test = train_test_split(features_scaled, target, test_size = 0.2, random_state = 6)

In [ ]:
x_Train.shape

In [ ]:
from sklearn import linear_model
model = sk.linear_model.LinearRegression()
model.fit(x_Train, y_Train)
y_pred = model.predict(x_Test)
mse = sk.metrics.mean_squared_error(y_Test, y_pred)
r2 = sk.metrics.r2_score(y_Test, y_pred)
print('Mean squared error: {:.2f}'.format(mse))

In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
x_Train = x_Train.reshape((x_Train.shape[0], x_Train.shape[1], 1))
x_Test = x_Test.reshape((x_Test.shape[0], x_Test.shape[1], 1))

In [ ]:


model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_Train.shape[1], 1)))
model.add(Conv1D(filters=128, kernel_size=1, activation='relu'))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='linear'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics='accuracy')
model.fit(x_Train, y_Train, epochs=100, batch_size=64)
model.evaluate(x_Test, y_Test)


In [ ]:
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(x_Train.shape[1],1)),
    layers.Dense(16, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.fit(x_Train, y_Train, epochs=100, batch_size=128)
model.evaluate(x_Test, y_Test)

In [ ]:
model = tf.keras.Sequential([
    layers.LSTM(64, return_sequences=True, input_shape=(x_Train.shape[1],1)),
    layers.LSTM(32),
    layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.fit(x_Train, y_Train, epochs=100, batch_size=32)
model.evaluate(x_Test, y_Test)